# Import libraries

In [1]:
import numpy as np
import timeit
import secrets
def timeitdec(func):
    def wrapper(*args, **kwargs):
        start_time = timeit.default_timer()
        result = func(*args, **kwargs)
        end_time = timeit.default_timer()
        elapsed_time = end_time - start_time
        print("Function {} took {:.6f} seconds to execute.".format(func.__name__, elapsed_time))
        return result
    return wrapper


# Key generation fun

In [57]:
@timeitdec
def generate_key(length: int = 1024) -> np.ndarray:
    key = secrets.token_bytes(10 * (length // 8))
    return np.unpackbits(np.frombuffer(key, dtype=np.uint8))[:10 * length].reshape(10, length)

# Creating cipher alg

In [58]:
@timeitdec
def whycipher(content: str, axkey: np.ndarray) -> str:
    iv = secrets.token_bytes(10)
    iv_bits = np.unpackbits(np.frombuffer(iv, dtype=np.uint8)).tolist()
    result = []
    for i, char in enumerate(content):
        key_index = i % 10
        binary_str = ''.join(map(str, axkey[key_index]))
        xor_input = int(binary_str, 2) ^ int(''.join(map(str, iv_bits[key_index * 8:(key_index + 1) * 8])), 2)
        ciphered_char = ord(char) ^ xor_input
        result.append(hex(ciphered_char)[2:].zfill(2))
    iv_hex = '|'.join(hex(byte)[2:].zfill(2) for byte in iv)
    return "|" + iv_hex + "|" + '|'.join(result) + '|'

# Creating decipher alg

In [59]:
@timeitdec
def dewhycipher(content: str, axkey: np.ndarray) -> str:
    parts = content.strip('|').split('|')
    iv = bytes(int(parts[i], 16) for i in range(0, 10))
    nums = parts[10:]
    iv_bits = np.unpackbits(np.frombuffer(iv, dtype=np.uint8)).tolist()

    result = []
    for i, num in enumerate(nums):
        if num:
            key_index = i % 10
            binary_str = ''.join(map(str, axkey[key_index]))
            xor_input = int(binary_str, 2) ^ int(''.join(map(str, iv_bits[key_index * 8:(key_index + 1) * 8])), 2)
            deciphered_char = int(num, 16) ^ xor_input
            if 0 <= deciphered_char <= 255:
                result.append(chr(deciphered_char))
    return ''.join(result)

# Using: 

In [60]:
key = generate_key()
text = "hello world"
cipher_text = whycipher(text, key)
decipher_text = dewhycipher(cipher_text, key)
print(cipher_text)
print(decipher_text)


Function generate_key took 0.000411 seconds to execute.
Function whycipher took 0.001594 seconds to execute.
Function dewhycipher took 0.001482 seconds to execute.
|d4|69|4c|95|78|84|e2|d6|71|13|1a285477389ce1a0e114ee1cc9952bf1d4ad432c4582cdaa2bd53a5e14442639fbfc802b26830f0a03dafc7a417c290e112f6b49b87d3580686f5419fca7f9ee3f05104726cd4f7ab2a7e2dbcdfb95082fda6bcd2b336f127dd20bc927e0c5afe6fc140e054c06799e644853b51ba1b45feda1ec9a69ea017cdcfe7b5ffa437c|3166ec5bc3d32ad93be36d4a822a9263a3d1611caac8853b4df31649a73d40a0e2c31c11b4e7d2c7c43311c50e805ca15c19029dcd51011db7720176267b9d905c1ff206ec4059c110f8d61e21a27e74303030613d5a287695995d1a20119a40aba6eef74d68f8b159570e5c4ca472a21f7e499551de27e49260631811131c0|d0de93f32cbdd49cce4e1825c5eca3136d6b883669ae56b196d11cc7c43b72953cd54fb7959dfc16b5d45078fea222d74ec78fd876d58213b802ff6985c78dcd805f0f2d8977ef9a4821edf5a43ad9e7a99e5121dc1a91e873a41e6d1ef4667bd9c5d4a144773ce5506eb7e21c1420b7f3c937f55973df6bf81e893cd60f6ab2|a297a0168b3f26d6ad66b74a2108435470c